In [6]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [7]:
print(module_path)

/Users/revilooliver/Documents/quantum_computing/research/pauli_sandwitching/cut4mitigation


In [8]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy
import circuit_cutter
import mlrecon_methods as ml
import numpy as np
import mthree

import qiskit
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

import qiskit.providers.aer.noise as noise
from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, total_counts, norm_dict
from utils.vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation, apply_checking_circuit
from mlrecon_methods import run_circuits, collect_fragment_circuits

In [9]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.22.2', 'qiskit-aer': '0.11.1', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.19.2', 'qiskit': '0.39.2', 'qiskit-nature': '0.5.1', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [10]:
numpy.set_printoptions(linewidth = 200)

simulation_backend = "qasm_simulator"

In [11]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-ornl', group='ornl', project='chm185')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_montreal') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_kolkata') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_mumbai') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_guadalupe') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-ornl', grou

In [12]:
device = provider.get_backend('ibm_auckland')
device

<IBMQBackend('ibm_auckland') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>

In [13]:
device.configuration().to_dict()['max_shots']

100000

In [14]:
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
noise_model = NoiseModel.from_backend(device, gate_error = False, thermal_relaxation= False)
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('measure', (4,)), ('measure', (5,)), ('measure', (6,)), ('measure', (7,)), ('measure', (8,)), ('measure', (9,)), ('measure', (10,)), ('measure', (11,)), ('measure', (12,)), ('measure', (13,)), ('measure', (14,)), ('measure', (15,)), ('measure', (16,)), ('measure', (17,)), ('measure', (18,)), ('measure', (19,)), ('measure', (20,)), ('measure', (21,)), ('measure', (22,)), ('measure', (23,)), ('measure', (24,)), ('measure', (25,)), ('measure', (26,))]


In [15]:
noisy_simulator = provider.get_backend('ibmq_qasm_simulator')

In [17]:
front_layer = QuantumCircuit.from_qasm_file("qasm/vqe_LiH_front.qasm")

In [18]:
mid_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_mid.qasm")

In [19]:
end_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_end.qasm")

In [20]:
orign_circuit = front_layer + mid_layer + end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_21113/743480212.py:1: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  orign_circuit = front_layer + mid_layer + end_layer


In [21]:
orign_circuit.draw()

┌───┐       ┌───────────────┐┌───────────────┐   ┌───────────────┐»
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Ry(0.0082611) ├»
            ├───┤       └┬──────────────┤├───────────────┤ │ └───────────────┘»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■─────────■────────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘           │        »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├────────────────────────────■────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                                     »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├──────────────────────────────────────»
     └─────────────────┘  └────────────┘                                      »
q_4: ─────────────────────────────────────────────────────────────────────────»
                                                                              »
«      ┌────────────┐                                    
«q_0: ─┤ Rz(1.3184) ├────────────────────────────────────
«     ┌┴────────────┴┐   ┌────────────┐                  
«q_1: ┤ Ry(0.013556) ├───┤ Rz(1.3353) ├──────────────────
«     └──────────────┘  ┌┴────────────┴┐ ┌──────────────┐
«q_2: ───────■──────────┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├
«            │        ┌─┴──────────────┴┐└┬────────────┬┘
«q_3: ───────■────────┤ Ry(-4.5556e-05) ├─┤ Rz(1.2199) ├─
«                     └─────────────────┘ └────────────┘ 
«q_4: ───────────────────────────────────────────────────
«

In [22]:
#checking circuit

In [23]:
phase_y_front = [0.008239051732521298, 0.013635804359443573, 0.05162548888524782, -2.1999413989625588e-05]
phase_z_front = [-0.010143905866716232, -0.028142425222093263, 0.008659821134257232, 1.4378429162081177]
phase_y_end = [0.008261085257040952, 0.013555948647231175, 0.0516520269898158, -4.555585169085397e-05]
phase_z_end = [1.3183908530236275, 1.3352666291100785, -0.26460520551963923, 1.2198754282688027]
x_front = [True, True, False, False]

In [25]:
qubit_id = 0
circuit = QuantumCircuit(5)
apply_checking_circuit(circuit, [qubit_id], [4], side = 'front', phase_y = phase_y_front[qubit_id], phase_z = phase_z_front[qubit_id], x = x_front[qubit_id])
circuit += orign_circuit
apply_checking_circuit(circuit, [qubit_id], [4], side = 'end', phase_y = phase_y_end[qubit_id], phase_z = phase_z_end[qubit_id])

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_21113/3972685507.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += orign_circuit


In [26]:
circuit.draw()

┌───┐       ┌───────────────┐┌───────────────┐   ┌──────────────┐»
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Rz(0.010144) ├»
            ├───┤       └┬──────────────┤├───────────────┤ │ └──────────────┘»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─┼─────────────────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘ │                 »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├──────────────────┼─────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                  │                 »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼─────────────────»
     └──────┬───┬──────┘  └────────────┘                   │                 »
q_4: ───────┤ H ├──────────────────────────────────────────■─────────────────»
            └───┘                                                            »
«     ┌────────────────┐┌───┐┌───┐┌───────────────┐┌───────────────┐   »
«q_0: ┤ Ry(-0.0082391) ├┤ X ├┤ X ├┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─»
«     └────────────────┘└───┘└───┘└───────────────┘└───────────────┘ │ »
«q_1: ───────────────────────────────────────────────────────────────■─»
«                                                                      »
«q_2: ─────────────────────────────────────────────────────────────────»
«                                                                      »
«q_3: ─────────────────────────────────────────────────────────────────»
«                                                                      »
«q_4: ─────────────────────────────────────────────────────────────────»
«                                                                      »
«     ┌───────────────┐ ┌────────────┐   ┌─────────────┐  ┌────────────────┐   »
«q_0: ┤ Ry(0.0082611) ├─┤ Rz(1.3184) ├───┤ Rz(-1.3184) ├──┤ Ry(-0.0082611) ├─■─»
«     └───────────────┘┌┴────────────┴┐  └┬────────────┤  └────────────────┘ │ »
«q_1: ────────■────────┤ Ry(0.013556) ├───┤ Rz(1.3353) ├─────────────────────┼─»
«             │        └──────────────┘  ┌┴────────────┴┐  ┌──────────────┐  │ »
«q_2: ────────■───────────────■──────────┤ Ry(0.051652) ├──┤ Rz(-0.26461) ├──┼─»
«                             │        ┌─┴──────────────┴┐ └┬────────────┬┘  │ »
«q_3: ────────────────────────■────────┤ Ry(-4.5556e-05) ├──┤ Rz(1.2199) ├───┼─»
«                                      └─────────────────┘  └────────────┘   │ »
«q_4: ───────────────────────────────────────────────────────────────────────■─»
«                                                                              »
«     ┌───────────────┐┌────────────┐
«q_0: ┤ Ry(0.0082611) ├┤ Rz(1.3184) ├
«     └───────────────┘└────────────┘
«q_1: ───────────────────────────────
«                                    
«q_2: ───────────────────────────────
«                                    
«q_3: ───────────────────────────────
«           ┌───┐                    
«q_4: ──────┤ H ├────────────────────
«           └───┘

In [28]:
pauli_list = read_from_file('LiH_hamiltonian_1.5.txt')
pauli_commute = find_commute_groups(pauli_list)

In [29]:
#generate individual measurment circuits that changes the basis
meas_qcs = []
for i in range(0, len(pauli_commute)):
    temp_qc = MeasureCircuit(pauli_commute[i], num_qubits = 4,num_qargs = 5)
    meas_qcs.append(temp_qc)
    print(temp_qc)

YYYY
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
XZYY
      ┌────────────┐ 
q_0: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_1: ────────────────
     ┌──────────────┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYXZ
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_2: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_3: ────────────────
                     
q_4: ────────────────
                     
XXYY
      ┌────────────┐ 
q_0: ─┤ U(π/2,0,π) ├─
      ├────────────┤ 
q_1: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYXX
     ┌─

In [30]:
circuit_list = []
for meas in meas_qcs:
    temp_circuit = circuit.copy()
    temp_circuit += meas
    circuit_list.append(temp_circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_21113/1717817760.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  temp_circuit += meas


In [31]:
circuit_list[-2].draw()

┌───┐       ┌───────────────┐┌───────────────┐   ┌──────────────┐»
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Rz(0.010144) ├»
            ├───┤       └┬──────────────┤├───────────────┤ │ └──────────────┘»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─┼─────────────────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘ │                 »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├──────────────────┼─────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                  │                 »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼─────────────────»
     └──────┬───┬──────┘  └────────────┘                   │                 »
q_4: ───────┤ H ├──────────────────────────────────────────■─────────────────»
            └───┘                                                            »
«     ┌────────────────┐┌───┐┌───┐┌───────────────┐┌───────────────┐   »
«q_0: ┤ Ry(-0.0082391) ├┤ X ├┤ X ├┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─»
«     └────────────────┘└───┘└───┘└───────────────┘└───────────────┘ │ »
«q_1: ───────────────────────────────────────────────────────────────■─»
«                                                                      »
«q_2: ─────────────────────────────────────────────────────────────────»
«                                                                      »
«q_3: ─────────────────────────────────────────────────────────────────»
«                                                                      »
«q_4: ─────────────────────────────────────────────────────────────────»
«                                                                      »
«     ┌───────────────┐ ┌────────────┐   ┌─────────────┐  ┌────────────────┐   »
«q_0: ┤ Ry(0.0082611) ├─┤ Rz(1.3184) ├───┤ Rz(-1.3184) ├──┤ Ry(-0.0082611) ├─■─»
«     └───────────────┘┌┴────────────┴┐  └┬────────────┤  └────────────────┘ │ »
«q_1: ────────■────────┤ Ry(0.013556) ├───┤ Rz(1.3353) ├─────────────────────┼─»
«             │        └──────────────┘  ┌┴────────────┴┐  ┌──────────────┐  │ »
«q_2: ────────■───────────────■──────────┤ Ry(0.051652) ├──┤ Rz(-0.26461) ├──┼─»
«                             │        ┌─┴──────────────┴┐ └┬────────────┬┘  │ »
«q_3: ────────────────────────■────────┤ Ry(-4.5556e-05) ├──┤ Rz(1.2199) ├───┼─»
«                                      └─────────────────┘  └────────────┘   │ »
«q_4: ───────────────────────────────────────────────────────────────────────■─»
«                                                                              »
«     ┌───────────────┐┌────────────┐
«q_0: ┤ Ry(0.0082611) ├┤ Rz(1.3184) ├
«     └───────────────┘└────────────┘
«q_1: ───────────────────────────────
«                                    
«q_2: ───────────────────────────────
«       ┌────────────┐               
«q_3: ──┤ U(π/2,0,π) ├───────────────
«       └───┬───┬────┘               
«q_4: ──────┤ H ├────────────────────
«           └───┘

In [32]:
# cuts = []
# for i in range(0, 1):
#     if i == 0 or i == qubits - 1:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 6)]
#     else:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 4)]
#     cuts += cut

In [33]:
cuts = [(circuit.qubits[0], 7),(circuit.qubits[0], 13)]

In [34]:
fragments_list = []
wire_path_map_list = []
for temp_qc in circuit_list:
    print(temp_qc)
    fragments, wire_path_map = circuit_cutter.cut_circuit(temp_qc, cuts)
    fragments_list.append(fragments)
    wire_path_map_list.append(wire_path_map)

            ┌───┐       ┌───────────────┐┌───────────────┐   ┌──────────────┐»
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Rz(0.010144) ├»
            ├───┤       └┬──────────────┤├───────────────┤ │ └──────────────┘»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─┼─────────────────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘ │                 »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├──────────────────┼─────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                  │                 »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼─────────────────»
     └──────┬───┬──────┘  └────────────┘                   │                 »
q_4: ───────┤ H ├──────────────────────────────────────────■─────────────────»
            └───┘                                                            »
«     ┌────────────────┐┌───┐┌───┐┌───────────────┐┌───────────────┐   »
«q_0: ┤ Ry(-0.0082391) ├┤ X ├┤ X ├┤ Ry(0.0082391) ├┤ Rz(-0

            ┌───┐       ┌───────────────┐┌───────────────┐   ┌──────────────┐»
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Rz(0.010144) ├»
            ├───┤       └┬──────────────┤├───────────────┤ │ └──────────────┘»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─┼─────────────────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘ │                 »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├──────────────────┼─────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                  │                 »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼─────────────────»
     └──────┬───┬──────┘  └────────────┘                   │                 »
q_4: ───────┤ H ├──────────────────────────────────────────■─────────────────»
            └───┘                                                            »
«     ┌────────────────┐┌───┐┌───┐┌───────────────┐┌───────────────┐   »
«q_0: ┤ Ry(-0.0082391) ├┤ X ├┤ X ├┤ Ry(0.0082391) ├┤ Rz(-0

            ┌───┐       ┌───────────────┐┌───────────────┐   ┌──────────────┐»
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Rz(0.010144) ├»
            ├───┤       └┬──────────────┤├───────────────┤ │ └──────────────┘»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─┼─────────────────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘ │                 »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├──────────────────┼─────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                  │                 »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼─────────────────»
     └──────┬───┬──────┘  └────────────┘                   │                 »
q_4: ───────┤ H ├──────────────────────────────────────────■─────────────────»
            └───┘                                                            »
«     ┌────────────────┐┌───┐┌───┐┌───────────────┐┌───────────────┐   »
«q_0: ┤ Ry(-0.0082391) ├┤ X ├┤ X ├┤ Ry(0.0082391) ├┤ Rz(-0

            ┌───┐       ┌───────────────┐┌───────────────┐   ┌──────────────┐»
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Rz(0.010144) ├»
            ├───┤       └┬──────────────┤├───────────────┤ │ └──────────────┘»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─┼─────────────────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘ │                 »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├──────────────────┼─────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                  │                 »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼─────────────────»
     └──────┬───┬──────┘  └────────────┘                   │                 »
q_4: ───────┤ H ├──────────────────────────────────────────■─────────────────»
            └───┘                                                            »
«     ┌────────────────┐┌───┐┌───┐┌───────────────┐┌───────────────┐   »
«q_0: ┤ Ry(-0.0082391) ├┤ X ├┤ X ├┤ Ry(0.0082391) ├┤ Rz(-0

            ┌───┐       ┌───────────────┐┌───────────────┐   ┌──────────────┐»
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Rz(0.010144) ├»
            ├───┤       └┬──────────────┤├───────────────┤ │ └──────────────┘»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─┼─────────────────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘ │                 »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├──────────────────┼─────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                  │                 »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼─────────────────»
     └──────┬───┬──────┘  └────────────┘                   │                 »
q_4: ───────┤ H ├──────────────────────────────────────────■─────────────────»
            └───┘                                                            »
«     ┌────────────────┐┌───┐┌───┐┌───────────────┐┌───────────────┐   »
«q_0: ┤ Ry(-0.0082391) ├┤ X ├┤ X ├┤ Ry(0.0082391) ├┤ Rz(-0

            ┌───┐       ┌───────────────┐┌───────────────┐   ┌──────────────┐»
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Rz(0.010144) ├»
            ├───┤       └┬──────────────┤├───────────────┤ │ └──────────────┘»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─┼─────────────────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘ │                 »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├──────────────────┼─────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                  │                 »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼─────────────────»
     └──────┬───┬──────┘  └────────────┘                   │                 »
q_4: ───────┤ H ├──────────────────────────────────────────■─────────────────»
            └───┘                                                            »
«     ┌────────────────┐┌───┐┌───┐┌───────────────┐┌───────────────┐   »
«q_0: ┤ Ry(-0.0082391) ├┤ X ├┤ X ├┤ Ry(0.0082391) ├┤ Rz(-0

            ┌───┐       ┌───────────────┐┌───────────────┐   ┌──────────────┐»
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Rz(0.010144) ├»
            ├───┤       └┬──────────────┤├───────────────┤ │ └──────────────┘»
q_1: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─┼─────────────────»
       ┌────┴───┴─────┐ ┌┴──────────────┤└───────────────┘ │                 »
q_2: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├──────────────────┼─────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                  │                 »
q_3: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼─────────────────»
     └──────┬───┬──────┘  └────────────┘                   │                 »
q_4: ───────┤ H ├──────────────────────────────────────────■─────────────────»
            └───┘                                                            »
«     ┌────────────────┐┌───┐┌───┐┌───────────────┐┌───────────────┐   »
«q_0: ┤ Ry(-0.0082391) ├┤ X ├┤ X ├┤ Ry(0.0082391) ├┤ Rz(-0

In [35]:
print(wire_path_map_list[0])

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(3, 'q'), 2)), (1, Qubit(QuantumRegister(4, 'q'), 0)), (0, Qubit(QuantumRegister(3, 'q'), 1))), Qubit(QuantumRegister(5, 'q'), 1): ((1, Qubit(QuantumRegister(4, 'q'), 3)),), Qubit(QuantumRegister(5, 'q'), 2): ((1, Qubit(QuantumRegister(4, 'q'), 1)),), Qubit(QuantumRegister(5, 'q'), 3): ((1, Qubit(QuantumRegister(4, 'q'), 2)),), Qubit(QuantumRegister(5, 'q'), 4): ((0, Qubit(QuantumRegister(3, 'q'), 0)),)}


In [36]:
total_variants = ml.fragment_variants(wire_path_map_list[0])

In [37]:
total_variants

24

In [38]:
shots = 10000 * total_variants

In [39]:
fragments_list[0][0].draw()

┌───┐                                                           »
q_0: ─────┤ H ├─────────────────────────────────────────■────────■────────»
     ┌────┴───┴────┐┌────────────────┐                  │        │        »
q_1: ┤ Rz(-1.3184) ├┤ Ry(-0.0082611) ├──────────────────┼────────■────────»
     └────┬───┬────┘├───────────────┬┘┌───────────────┐ │ ┌──────────────┐»
q_2: ─────┤ X ├─────┤ Ry(0.0082391) ├─┤ Rz(-0.010144) ├─■─┤ Rz(0.010144) ├»
          └───┘     └───────────────┘ └───────────────┘   └──────────────┘»
«           ┌───┐                                     
«q_0: ──────┤ H ├─────────────────────────────────────
«     ┌─────┴───┴─────┐ ┌────────────┐┌──────────────┐
«q_1: ┤ Ry(0.0082611) ├─┤ Rz(1.3184) ├┤ U(π/2,0,π/2) ├
«     ├───────────────┴┐└───┬───┬────┘└──────────────┘
«q_2: ┤ Ry(-0.0082391) ├────┤ X ├─────────────────────
«     └────────────────┘    └───┘

In [40]:
fragment_cuts_list = []
for index in range(0, len(fragments_list)):
    for idx, fragment in enumerate(fragments_list[index]):
        print(f"fragment {idx}:")
        print(fragment)
        print()
    fragment_cuts = ml.fragment_cuts(wire_path_map_list[index])
    fragment_cuts_list.append(fragment_cuts)


fragment 0:
          ┌───┐                                                           »
q_0: ─────┤ H ├─────────────────────────────────────────■────────■────────»
     ┌────┴───┴────┐┌────────────────┐                  │        │        »
q_1: ┤ Rz(-1.3184) ├┤ Ry(-0.0082611) ├──────────────────┼────────■────────»
     └────┬───┬────┘├───────────────┬┘┌───────────────┐ │ ┌──────────────┐»
q_2: ─────┤ X ├─────┤ Ry(0.0082391) ├─┤ Rz(-0.010144) ├─■─┤ Rz(0.010144) ├»
          └───┘     └───────────────┘ └───────────────┘   └──────────────┘»
«           ┌───┐                                     
«q_0: ──────┤ H ├─────────────────────────────────────
«     ┌─────┴───┴─────┐ ┌────────────┐┌──────────────┐
«q_1: ┤ Ry(0.0082611) ├─┤ Rz(1.3184) ├┤ U(π/2,0,π/2) ├
«     ├───────────────┴┐└───┬───┬────┘└──────────────┘
«q_2: ┤ Ry(-0.0082391) ├────┤ X ├─────────────────────
«     └────────────────┘    └───┘                     

fragment 1:
            ┌───┐       ┌───────────────┐┌───────────────┐

In [41]:
print(fragment_cuts_list[-1])

[{'prep': 1, 'meas': 1}, {'prep': 1, 'meas': 1}]


In [42]:
wire_path_map

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(3, 'q'), 2)),
  (1, Qubit(QuantumRegister(4, 'q'), 0)),
  (0, Qubit(QuantumRegister(3, 'q'), 1))),
 Qubit(QuantumRegister(5, 'q'), 1): ((1, Qubit(QuantumRegister(4, 'q'), 3)),),
 Qubit(QuantumRegister(5, 'q'), 2): ((1, Qubit(QuantumRegister(4, 'q'), 1)),),
 Qubit(QuantumRegister(5, 'q'), 3): ((1, Qubit(QuantumRegister(4, 'q'), 2)),),
 Qubit(QuantumRegister(5, 'q'), 4): ((0, Qubit(QuantumRegister(3, 'q'), 0)),)}

In [43]:
num_fragments = len(fragments)
num_fragments

2

In [44]:
num_tomo_circuits = len(meas_qcs)
num_tomo_circuits

25

In [45]:
fragments[0].draw()

┌───┐                                                           »
q_0: ─────┤ H ├─────────────────────────────────────────■────────■────────»
     ┌────┴───┴────┐┌────────────────┐                  │        │        »
q_1: ┤ Rz(-1.3184) ├┤ Ry(-0.0082611) ├──────────────────┼────────■────────»
     └────┬───┬────┘├───────────────┬┘┌───────────────┐ │ ┌──────────────┐»
q_2: ─────┤ X ├─────┤ Ry(0.0082391) ├─┤ Rz(-0.010144) ├─■─┤ Rz(0.010144) ├»
          └───┘     └───────────────┘ └───────────────┘   └──────────────┘»
«           ┌───┐                     
«q_0: ──────┤ H ├─────────────────────
«     ┌─────┴───┴─────┐ ┌────────────┐
«q_1: ┤ Ry(0.0082611) ├─┤ Rz(1.3184) ├
«     ├───────────────┴┐└───┬───┬────┘
«q_2: ┤ Ry(-0.0082391) ├────┤ X ├─────
«     └────────────────┘    └───┘

In [59]:
fragments[1].draw()

┌───┐       ┌───────────────┐┌───────────────┐   ┌───────────────┐»
q_0: ───────┤ X ├───────┤ Ry(0.0082391) ├┤ Rz(-0.010144) ├─■─┤ Ry(0.0082611) ├»
       ┌────┴───┴─────┐ ├───────────────┤└───────────────┘ │ └───────────────┘»
q_1: ──┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├──────────────────┼─────────■────────»
     ┌─┴──────────────┴┐└─┬────────────┬┘                  │         │        »
q_2: ┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├───────────────────┼─────────┼────────»
     └──────┬───┬──────┘ ┌┴────────────┴┐┌───────────────┐ │         │        »
q_3: ───────┤ X ├────────┤ Ry(0.013636) ├┤ Rz(-0.028142) ├─■─────────■────────»
            └───┘        └──────────────┘└───────────────┘                    »
«      ┌────────────┐                                    
«q_0: ─┤ Rz(1.3184) ├────────────────────────────────────
«      └────────────┘   ┌──────────────┐ ┌──────────────┐
«q_1: ───────■──────────┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├
«            │        ┌─┴──────────────┴┐└┬────────────┬┘
«q_2: ───────■────────┤ Ry(-4.5556e-05) ├─┤ Rz(1.2199) ├─
«     ┌──────────────┐└──┬────────────┬─┘ └────────────┘ 
«q_3: ┤ Ry(0.013556) ├───┤ Rz(1.3353) ├──────────────────
«     └──────────────┘   └────────────┘

In [60]:
hardware_index = 1

In [61]:
print(shots)

240000


In [62]:
tomography_circuits = []
for idx in range(0, len(fragments_list)):
    fragments = fragments_list[idx]
    wire_path_map = wire_path_map_list[idx]
    temp_frag_circuits = [0] * len(fragments)
    for i in range(0, len(fragments)):
        if i == hardware_index:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
        else:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
    tomography_circuits.append(temp_frag_circuits)

In [63]:
#contains 34 basis
len(tomography_circuits)

25

In [64]:
#contains two fragments
len(tomography_circuits[0])

2

In [65]:
len(tomography_circuits[0][1])

1

In [66]:
#tomography circuits
len(tomography_circuits[0][1][0])

12

In [67]:
multi_exp_circuits = [[],[]]
for i in range(0, len(tomography_circuits)):
    #for each basis collect the three fragments:
    for j in range(0, num_fragments):
        fragment_circ = tomography_circuits[i][j][0]
        multi_exp_circuits[j] += fragment_circ

In [68]:
#16,14,11,8
#16,11,8,14,

In [70]:
multi_exp_circuits[1][-200].draw()

┌─────────────────┐ ░        ┌───┐       ┌───────────────┐»
  q_0: ┤ U3(-1.9106,π,0) ├─░────────┤ X ├───────┤ Ry(0.0082391) ├»
       └─────────────────┘ ░   ┌────┴───┴─────┐ ├───────────────┤»
  q_1: ────────────────────░───┤ Ry(0.051625) ├─┤ Rz(0.0086598) ├»
                           ░ ┌─┴──────────────┴┐└─┬────────────┬┘»
  q_2: ────────────────────░─┤ Ry(-2.1999e-05) ├──┤ Rz(1.4378) ├─»
                           ░ └──────┬───┬──────┘ ┌┴────────────┴┐»
  q_3: ────────────────────░────────┤ X ├────────┤ Ry(0.013636) ├»
                           ░        └───┘        └──────────────┘»
c67_0: ══════════════════════════════════════════════════════════»
                                                                 »
c67_1: ══════════════════════════════════════════════════════════»
                                                                 »
c67_2: ══════════════════════════════════════════════════════════»
                                                                 »
c67_3: ══════════════════════════════════════════════════════════»
                                                                 »
«       ┌───────────────┐   ┌───────────────┐ ┌────────────┐ »
«  q_0: ┤ Rz(-0.010144) ├─■─┤ Ry(0.0082611) ├─┤ Rz(1.3184) ├─»
«       └───────────────┘ │ └───────────────┘ └────────────┘ »
«  q_1: ──────────────────┼─────────■───────────────■────────»
«                         │         │               │        »
«  q_2: ──────────────────┼─────────┼───────────────■────────»
«       ┌───────────────┐ │         │        ┌──────────────┐»
«  q_3: ┤ Rz(-0.028142) ├─■─────────■────────┤ Ry(0.013556) ├»
«       └───────────────┘                    └──────────────┘»
«c67_0: ═════════════════════════════════════════════════════»
«                                                            »
«c67_1: ═════════════════════════════════════════════════════»
«                                                            »
«c67_2: ═════════════════════════════════════════════════════»
«                                                            »
«c67_3: ═════════════════════════════════════════════════════»
«                                                            »
«                                                           ░ ┌───┐      ┌─┐
«  q_0: ────────────────────────────────────────────────────░─┤ H ├──────┤M├
«         ┌──────────────┐ ┌──────────────┐┌──────────────┐ ░ └┬─┬┘      └╥┘
«  q_1: ──┤ Ry(0.051652) ├─┤ Rz(-0.26461) ├┤ U(π/2,0,π/2) ├─░──┤M├────────╫─
«       ┌─┴──────────────┴┐└┬────────────┬┘├──────────────┤ ░  └╥┘ ┌─┐    ║ 
«  q_2: ┤ Ry(-4.5556e-05) ├─┤ Rz(1.2199) ├─┤ U(π/2,0,π/2) ├─░───╫──┤M├────╫─
«       └──┬────────────┬─┘ └────────────┘ └──────────────┘ ░   ║  └╥┘┌─┐ ║ 
«  q_3: ───┤ Rz(1.3353) ├───────────────────────────────────░───╫───╫─┤M├─╫─
«          └────────────┘                                   ░   ║   ║ └╥┘ ║ 
«c67_0: ════════════════════════════════════════════════════════╬═══╬══╬══╩═
«                                                               ║   ║  ║    
«c67_1: ════════════════════════════════════════════════════════╩═══╬══╬════
«                                                                   ║  ║    
«c67_2: ════════════════════════════════════════════════════════════╩══╬════
«                                                                      ║    
«c67_3: ═══════════════════════════════════════════════════════════════╩════
«

In [102]:
frag_datas = [0] * num_fragments
for i in range(0, num_fragments):
    if i == hardware_index:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = [16,11,8,14],
                                     backend = device, shots = shots // total_variants, 
                                     optimization_level = 3, monitor_jobs = True)
    else:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = None,
                                     backend = simulation_backend, shots = shots // total_variants,
                                     optimization_level = 3, monitor_jobs = True, noise_model = None)
       

job_id:  bd8b3cb0-6dd4-4ea5-943e-ec96774344e0
Job Status: job has successfully run
job_id:  6391891344034d72ac4d4cd5
Job Status: job has successfully run


In [103]:
frag_datas[0]

[Result(backend_name='qasm_simulator', backend_version='0.11.1', qobj_id='79c2001b-77ff-47b6-a390-899cad45bd89', job_id='bd8b3cb0-6dd4-4ea5-943e-ec96774344e0', success=True, results=[ExperimentResult(shots=10000, success=True, meas_level=2, data=ExperimentResultData(counts={'0x1': 5063, '0x6': 1, '0x3': 4936}), header=QobjExperimentHeader(clbit_labels=[['c50', 0], ['c50', 1], ['c50', 2]], creg_sizes=[['c50', 3]], global_phase=6.282157874422679, memory_slots=3, metadata={}, n_qubits=3, name="(('S0', 'S0', 'S0'), ('Z', 'Z', 'Z'))", qreg_sizes=[['q', 3]], qubit_labels=[['q', 0], ['q', 1], ['q', 2]]), status=DONE, seed_simulator=2802088226, metadata={'parallel_state_update': 8, 'parallel_shots': 1, 'sample_measure_time': 0.003095391, 'noise': 'ideal', 'batched_shots_optimization': False, 'remapped_qubits': False, 'device': 'CPU', 'active_input_qubits': [0, 1, 2], 'measure_sampling': True, 'num_clbits': 3, 'input_qubit_map': [[2, 2], [1, 1], [0, 0]], 'num_qubits': 3, 'method': 'statevector'

In [100]:
# from qiskit.providers.ibmq.managed import IBMQJobManager
# job_manager = IBMQJobManager()

In [101]:
# job_set = job_manager.retrieve_job_set(job_set_id='973c2fdceaf547389a770d19aad463d8-1669966038684177', provider=provider)

In [68]:
# managed_results = job_set.results()

In [104]:
frag_datas1_copy = frag_datas[1].copy()

In [105]:
frag_datas[1] = [frag_datas[1][0].combine_results()]

In [106]:
# frag_datas[0] = run_circuits(multi_exp_circuits[0], initial_layout = None,
#                              backend = simulation_backend, shots = shots // total_variants,
#                              optimization_level = 3, monitor_jobs = True, noise_model = None)

In [105]:
frag_datas[0][0].results[0].header.name

"(('S0', 'S0', 'S0'), ('Z', 'Z', 'Z'))"

In [106]:
num_tomo_circuits

25

In [107]:
test = [[], []]

In [108]:
results_list = [[], []]
counts_list = [[], []] 
for i in range(0, num_fragments): 
    for j in range(0, num_tomo_circuits):
        start_idx = j * 12
        end_idx = (j + 1) * 12
        results_list[i].append(frag_datas[i][0].results[start_idx : end_idx])
        counts_list[i].append(frag_datas[i][0].get_counts()[start_idx : end_idx])


In [109]:
frag_datas[1][0].get_counts()

[{'0000': 22,
  '0001': 1261,
  '0010': 46,
  '0011': 1129,
  '0100': 38,
  '0101': 1238,
  '0110': 32,
  '0111': 1175,
  '1000': 39,
  '1001': 1269,
  '1010': 31,
  '1011': 1164,
  '1100': 24,
  '1101': 1247,
  '1110': 27,
  '1111': 1258},
 {'0000': 682,
  '0001': 575,
  '0010': 615,
  '0011': 533,
  '0100': 673,
  '0101': 567,
  '0110': 646,
  '0111': 579,
  '1000': 636,
  '1001': 654,
  '1010': 641,
  '1011': 616,
  '1100': 692,
  '1101': 612,
  '1110': 657,
  '1111': 622},
 {'0000': 791,
  '0001': 507,
  '0010': 751,
  '0011': 439,
  '0100': 763,
  '0101': 480,
  '0110': 695,
  '0111': 456,
  '1000': 820,
  '1001': 503,
  '1010': 768,
  '1011': 494,
  '1100': 788,
  '1101': 512,
  '1110': 770,
  '1111': 463},
 {'0000': 612,
  '0001': 606,
  '0010': 610,
  '0011': 568,
  '0100': 627,
  '0101': 586,
  '0110': 631,
  '0111': 552,
  '1000': 768,
  '1001': 600,
  '1010': 742,
  '1011': 538,
  '1100': 735,
  '1101': 564,
  '1110': 688,
  '1111': 573},
 {'0000': 496,
  '0001': 832,
  '001

In [110]:
frag_datas[0][0].get_counts()

[{'001': 5063, '110': 1, '011': 4936},
 {'111': 2398, '001': 2574, '000': 1, '011': 2474, '101': 2553},
 {'001': 2475, '011': 2520, '101': 2533, '111': 2472},
 {'100': 1, '001': 1625, '000': 3233, '011': 1660, '010': 3481},
 {'101': 823,
  '011': 820,
  '111': 860,
  '100': 1632,
  '110': 1658,
  '001': 856,
  '010': 1729,
  '000': 1622},
 {'011': 853,
  '111': 837,
  '010': 1715,
  '000': 1651,
  '101': 856,
  '001': 856,
  '100': 1620,
  '110': 1612},
 {'001': 1692, '000': 3272, '010': 3374, '011': 1662},
 {'101': 846,
  '100': 1671,
  '110': 1702,
  '001': 851,
  '111': 835,
  '011': 765,
  '010': 1722,
  '000': 1608},
 {'100': 1655,
  '110': 1668,
  '001': 850,
  '011': 793,
  '111': 793,
  '010': 1695,
  '000': 1728,
  '101': 818},
 {'010': 3305, '011': 1711, '000': 3260, '001': 1724},
 {'011': 810,
  '111': 805,
  '101': 866,
  '001': 806,
  '110': 1670,
  '100': 1687,
  '010': 1661,
  '000': 1695},
 {'101': 914,
  '001': 818,
  '110': 1648,
  '100': 1673,
  '011': 818,
  '111': 

In [111]:
from mlrecon_methods import organize_tomography_data_from_list
organized_data = []
for i in range(0, num_tomo_circuits):
    frag_data = []
    frag_targets = ml.identify_frag_targets(wire_path_map_list[i])
    for j in range(0, num_fragments):
        frag_data.append(organize_tomography_data_from_list(results_list[j][i], counts_list[j][i],
                                              frag_targets[j].get("prep"),
                                              frag_targets[j].get("meas"),
                                              prep_basis = "SIC", extra_qc = []))
    organized_data.append(frag_data)

(('S0', 'S0', 'S0'), ('Z', 'Z', 'Z')) ('S0', 'S0', 'S0') ('Z', 'Z', 'Z')
(('S0', 'S0', 'S0'), ('Z', 'Z', 'Z')) ('S0',) ('Z',) [1] [2]
(('S0', 'S0', 'S0'), ('Z', 'Z', 'X')) ('S0', 'S0', 'S0') ('Z', 'Z', 'X')
(('S0', 'S0', 'S0'), ('Z', 'Z', 'X')) ('S0',) ('X',) [1] [2]
(('S0', 'S0', 'S0'), ('Z', 'Z', 'Y')) ('S0', 'S0', 'S0') ('Z', 'Z', 'Y')
(('S0', 'S0', 'S0'), ('Z', 'Z', 'Y')) ('S0',) ('Y',) [1] [2]
(('S0', 'S1', 'S0'), ('Z', 'Z', 'Z')) ('S0', 'S1', 'S0') ('Z', 'Z', 'Z')
(('S0', 'S1', 'S0'), ('Z', 'Z', 'Z')) ('S1',) ('Z',) [1] [2]
(('S0', 'S1', 'S0'), ('Z', 'Z', 'X')) ('S0', 'S1', 'S0') ('Z', 'Z', 'X')
(('S0', 'S1', 'S0'), ('Z', 'Z', 'X')) ('S1',) ('X',) [1] [2]
(('S0', 'S1', 'S0'), ('Z', 'Z', 'Y')) ('S0', 'S1', 'S0') ('Z', 'Z', 'Y')
(('S0', 'S1', 'S0'), ('Z', 'Z', 'Y')) ('S1',) ('Y',) [1] [2]
(('S0', 'S2', 'S0'), ('Z', 'Z', 'Z')) ('S0', 'S2', 'S0') ('Z', 'Z', 'Z')
(('S0', 'S2', 'S0'), ('Z', 'Z', 'Z')) ('S2',) ('Z',) [1] [2]
(('S0', 'S2', 'S0'), ('Z', 'Z', 'X')) ('S0', 'S2', 'S0') ('Z',

In [112]:
for final_bits, fixed_bit_data in frag_data[0].items():
    prep_meas_states, state_counts = zip(*fixed_bit_data.items())
    prep_labels, meas_labels = zip(*prep_meas_states)
    prep_qubit_num = len(prep_labels[0])
    meas_qubit_num = len(meas_labels[0])
    print(prep_qubit_num)

1
1
1


In [113]:
len(organized_data)

25

In [114]:
organized_data[2]

[{'11': {(('S0',), ('Zp',)): 4952,
   (('S0',), ('Xp',)): 2477,
   (('S0',), ('Xm',)): 2493,
   (('S0',), ('Ym',)): 2480,
   (('S0',), ('Yp',)): 2467,
   (('S1',), ('Zp',)): 1579,
   (('S1',), ('Xm',)): 830,
   (('S1',), ('Xp',)): 811,
   (('S1',), ('Yp',)): 824,
   (('S1',), ('Ym',)): 878,
   (('S2',), ('Zp',)): 1695,
   (('S2',), ('Xm',)): 825,
   (('S2',), ('Xp',)): 775,
   (('S2',), ('Ym',)): 763,
   (('S2',), ('Yp',)): 775,
   (('S3',), ('Zm',)): 1,
   (('S3',), ('Zp',)): 1718,
   (('S3',), ('Xp',)): 825,
   (('S3',), ('Xm',)): 819,
   (('S3',), ('Ym',)): 807,
   (('S3',), ('Yp',)): 850,
   (('S0',), ('Zm',)): 0,
   (('S1',), ('Zm',)): 0,
   (('S2',), ('Zm',)): 0},
  '01': {(('S0',), ('Zp',)): 5048,
   (('S0',), ('Xm',)): 2476,
   (('S0',), ('Xp',)): 2553,
   (('S0',), ('Ym',)): 2543,
   (('S0',), ('Yp',)): 2510,
   (('S1',), ('Zp',)): 1763,
   (('S1',), ('Xp',)): 865,
   (('S1',), ('Xm',)): 818,
   (('S1',), ('Ym',)): 819,
   (('S1',), ('Yp',)): 853,
   (('S2',), ('Zp',)): 1615,


In [115]:
organized_data[-1][-1]

{'000': {(('S0',), ('Zp',)): 4,
  (('S0',), ('Zm',)): 225,
  (('S0',), ('Xp',)): 112,
  (('S0',), ('Xm',)): 103,
  (('S0',), ('Yp',)): 127,
  (('S0',), ('Ym',)): 100,
  (('S1',), ('Zp',)): 146,
  (('S1',), ('Zm',)): 78,
  (('S1',), ('Xp',)): 88,
  (('S1',), ('Xm',)): 140,
  (('S1',), ('Yp',)): 68,
  (('S1',), ('Ym',)): 164,
  (('S2',), ('Zp',)): 162,
  (('S2',), ('Zm',)): 75,
  (('S2',), ('Xp',)): 126,
  (('S2',), ('Xm',)): 76,
  (('S2',), ('Yp',)): 110,
  (('S2',), ('Ym',)): 106,
  (('S3',), ('Zp',)): 184,
  (('S3',), ('Zm',)): 80,
  (('S3',), ('Xp',)): 99,
  (('S3',), ('Xm',)): 147,
  (('S3',), ('Yp',)): 139,
  (('S3',), ('Ym',)): 82},
 '001': {(('S0',), ('Zm',)): 2,
  (('S0',), ('Xp',)): 1,
  (('S0',), ('Xm',)): 3,
  (('S0',), ('Ym',)): 3,
  (('S1',), ('Zp',)): 2,
  (('S1',), ('Zm',)): 1,
  (('S1',), ('Xp',)): 2,
  (('S1',), ('Xm',)): 1,
  (('S1',), ('Yp',)): 1,
  (('S2',), ('Zp',)): 3,
  (('S2',), ('Xp',)): 3,
  (('S2',), ('Xm',)): 1,
  (('S2',), ('Ym',)): 2,
  (('S3',), ('Zp',)): 

In [116]:
recombined_lists = []
filtered_recombined_lists = []
norm_filtered_lists = []
for i in range(0, num_tomo_circuits):
    direct_models = ml.direct_fragment_model(organized_data[i])
    likely_models = ml.maximum_likelihood_model(direct_models)

    direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map_list[i])
    likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map_list[i])
    recombined_lists.append(likely_recombined_dist)
    filtered_recombined_lists.append(filter_results(likely_recombined_dist, [0]))
    norm_filtered_lists.append(norm_dict(filter_results(likely_recombined_dist, [0])))

{'01': {(('S0',), ('Zp',)): 5063, (('S0',), ('Xp',)): 2574, (('S0',), ('Xm',)): 2553, (('S0',), ('Yp',)): 2475, (('S0',), ('Ym',)): 2533, (('S1',), ('Zp',)): 1625, (('S1',), ('Xm',)): 823, (('S1',), ('Xp',)): 856, (('S1',), ('Ym',)): 856, (('S1',), ('Yp',)): 856, (('S2',), ('Zp',)): 1692, (('S2',), ('Xm',)): 846, (('S2',), ('Xp',)): 851, (('S2',), ('Yp',)): 850, (('S2',), ('Ym',)): 818, (('S3',), ('Zp',)): 1724, (('S3',), ('Xm',)): 866, (('S3',), ('Xp',)): 806, (('S3',), ('Ym',)): 914, (('S3',), ('Yp',)): 818, (('S0',), ('Zm',)): 0, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '10': {(('S0',), ('Zm',)): 1, (('S1',), ('Zp',)): 3481, (('S1',), ('Xm',)): 1658, (('S1',), ('Xp',)): 1729, (('S1',), ('Yp',)): 1715, (('S1',), ('Ym',)): 1612, (('S2',), ('Zp',)): 3374, (('S2',), ('Xm',)): 1702, (('S2',), ('Xp',)): 1722, (('S2',), ('Ym',)): 1668, (('S2',), ('Yp',)): 1695, (('S3',), ('Zp',)): 3305, (('S3',), ('Xm',)): 1670, (('S3',), ('Xp',)): 1661, (('S3',), ('Ym',)): 164

{'000': {(('S0',), ('Zp',)): 149, (('S0',), ('Zm',)): 4619, (('S0',), ('Xp',)): 2450, (('S0',), ('Xm',)): 2281, (('S0',), ('Yp',)): 2943, (('S0',), ('Ym',)): 1915, (('S1',), ('Zp',)): 2523, (('S1',), ('Zm',)): 2209, (('S1',), ('Xp',)): 1880, (('S1',), ('Xm',)): 2923, (('S1',), ('Yp',)): 169, (('S1',), ('Ym',)): 4599, (('S2',), ('Zp',)): 3320, (('S2',), ('Zm',)): 1361, (('S2',), ('Xp',)): 4284, (('S2',), ('Xm',)): 320, (('S2',), ('Yp',)): 2701, (('S2',), ('Ym',)): 2013, (('S3',), ('Zp',)): 3410, (('S3',), ('Zm',)): 1353, (('S3',), ('Xp',)): 780, (('S3',), ('Xm',)): 3943, (('S3',), ('Yp',)): 3645, (('S3',), ('Ym',)): 1124}, '001': {(('S0',), ('Zp',)): 1, (('S0',), ('Zm',)): 37, (('S0',), ('Xp',)): 24, (('S0',), ('Xm',)): 17, (('S0',), ('Yp',)): 38, (('S0',), ('Ym',)): 21, (('S1',), ('Zp',)): 20, (('S1',), ('Zm',)): 15, (('S1',), ('Xp',)): 12, (('S1',), ('Xm',)): 27, (('S1',), ('Yp',)): 2, (('S1',), ('Ym',)): 41, (('S2',), ('Zp',)): 32, (('S2',), ('Zm',)): 15, (('S2',), ('Xp',)): 47, (('S

{'10': {(('S0',), ('Zm',)): 1, (('S0',), ('Yp',)): 1, (('S1',), ('Zp',)): 3384, (('S1',), ('Xm',)): 1631, (('S1',), ('Xp',)): 1702, (('S1',), ('Yp',)): 1672, (('S1',), ('Ym',)): 1649, (('S2',), ('Zp',)): 3418, (('S2',), ('Xm',)): 1615, (('S2',), ('Xp',)): 1725, (('S2',), ('Ym',)): 1618, (('S2',), ('Yp',)): 1712, (('S3',), ('Zp',)): 3335, (('S3',), ('Xm',)): 1631, (('S3',), ('Xp',)): 1660, (('S3',), ('Ym',)): 1663, (('S3',), ('Yp',)): 1681, (('S0',), ('Zp',)): 0, (('S0',), ('Xp',)): 0, (('S0',), ('Xm',)): 0, (('S0',), ('Ym',)): 0, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '00': {(('S0',), ('Zp',)): 1, (('S0',), ('Xm',)): 1, (('S0',), ('Xp',)): 1, (('S1',), ('Zp',)): 3245, (('S1',), ('Xm',)): 1666, (('S1',), ('Xp',)): 1642, (('S1',), ('Yp',)): 1632, (('S1',), ('Ym',)): 1652, (('S2',), ('Zp',)): 3335, (('S2',), ('Xm',)): 1631, (('S2',), ('Xp',)): 1669, (('S2',), ('Ym',)): 1730, (('S2',), ('Yp',)): 1618, (('S3',), ('Zp',)): 3152, (('S3',), ('Xm',)): 1665, (('S3'

{'000': {(('S0',), ('Zp',)): 72, (('S0',), ('Zm',)): 5003, (('S0',), ('Xp',)): 2685, (('S0',), ('Xm',)): 2495, (('S0',), ('Yp',)): 2786, (('S0',), ('Ym',)): 2275, (('S1',), ('Zp',)): 3013, (('S1',), ('Zm',)): 1894, (('S1',), ('Xp',)): 1881, (('S1',), ('Xm',)): 3119, (('S1',), ('Yp',)): 257, (('S1',), ('Ym',)): 4653, (('S2',), ('Zp',)): 3366, (('S2',), ('Zm',)): 1648, (('S2',), ('Xp',)): 4820, (('S2',), ('Xm',)): 296, (('S2',), ('Yp',)): 3028, (('S2',), ('Ym',)): 1899, (('S3',), ('Zp',)): 3465, (('S3',), ('Zm',)): 1607, (('S3',), ('Xp',)): 888, (('S3',), ('Xm',)): 4125, (('S3',), ('Yp',)): 4056, (('S3',), ('Ym',)): 963}, '001': {(('S0',), ('Zm',)): 49, (('S0',), ('Xp',)): 19, (('S0',), ('Xm',)): 15, (('S0',), ('Yp',)): 20, (('S0',), ('Ym',)): 17, (('S1',), ('Zp',)): 22, (('S1',), ('Zm',)): 23, (('S1',), ('Xp',)): 14, (('S1',), ('Xm',)): 34, (('S1',), ('Yp',)): 2, (('S1',), ('Ym',)): 38, (('S2',), ('Zp',)): 30, (('S2',), ('Zm',)): 13, (('S2',), ('Xp',)): 35, (('S2',), ('Xm',)): 2, (('S2'

In [117]:
# direct_models = ml.direct_fragment_model(frag_data)
# likely_models = ml.maximum_likelihood_model(direct_models)

# direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map)
# likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map)

In [118]:
direct_recombined_dist

{'10000': 0.00030021640625653537,
 '10100': 4.5676789742681245e-05,
 '11000': -1.163590852240748e-05,
 '10010': 0.037608371375842095,
 '10110': 0.0006071993440306032,
 '11010': -3.07836151072391e-05,
 '11110': 5.262129612575716e-05,
 '10001': 1.043967158253291e-06,
 '10101': 7.577136834321203e-08,
 '11001': 4.362254602571582e-09,
 '10011': -7.783058154777706e-06,
 '10111': 1.1724386481773413e-07,
 '11011': 3.0481723855687777e-07,
 '11111': 1.1810170139884186e-10,
 '00001': 0.022112739586840433,
 '00101': 0.0002522201201487126,
 '01001': 0.0002808105551712756,
 '00011': 0.9192108738900511,
 '00111': 0.009135233205515776,
 '01011': 0.010293812994882016,
 '01111': 0.00014888073719118258}

In [119]:
likely_recombined_dist

{'10000': 0.0008484383877559469,
 '10100': 2.5262722790984767e-06,
 '11000': 5.79361707646987e-06,
 '10010': 0.03289842348866125,
 '10110': 0.0007731490537249919,
 '11010': 0.00043213720685947267,
 '00001': 0.021777535273215288,
 '00101': 1.957771391282202e-05,
 '01001': 5.9189530751007324e-05,
 '00011': 0.9238006439109681,
 '00111': 0.00902511296215247,
 '01011': 0.010357472582643119}

In [120]:
filtered_recombined_lists

[{'0001': 0.06315652949251488,
  '0101': 0.05683701631761239,
  '1001': 0.06144929433729626,
  '1101': 0.05871206305237556,
  '0011': 0.06351531568676447,
  '0111': 0.05923309466181117,
  '1011': 0.06290293148446084,
  '1111': 0.06245200805146483,
  '0000': 0.06125433894534413,
  '0100': 0.05513599598892019,
  '1000': 0.05959227334588689,
  '1100': 0.056991482901551316,
  '0010': 0.06173780801488109,
  '0110': 0.057544890043600846,
  '1010': 0.06106254161785733,
  '1110': 0.060666904493319025},
 {'0000': 0.002422450053275579,
  '0100': 0.00224340176035162,
  '1000': 0.0018618874670435553,
  '1100': 0.002088671303027721,
  '0010': 0.11545247757418516,
  '0110': 0.11789245138774403,
  '1010': 0.11466311866495187,
  '1110': 0.11862278838044134,
  '0001': 0.0024758398013981203,
  '0101': 0.002258629410433687,
  '1001': 0.0018891192920226475,
  '1101': 0.0021179895590832474,
  '0011': 0.11732869801920805,
  '0111': 0.12001220012602912,
  '1011': 0.11650825750193601,
  '1111': 0.120591404613

In [121]:
from utils.utils import filter_results

In [122]:
filter_direct_recombined = filter_results(direct_recombined_dist, [0])

In [123]:
filter_direct_recombined

{'0001': 0.022112739586840433,
 '0101': 0.0002522201201487126,
 '1001': 0.0002808105551712756,
 '0011': 0.9192108738900511,
 '0111': 0.009135233205515776,
 '1011': 0.010293812994882016,
 '1111': 0.00014888073719118258}

In [124]:

filter_likely_recombined = filter_results(likely_recombined_dist, [0])

In [125]:
filter_likely_recombined

{'0001': 0.021777535273215288,
 '0101': 1.957771391282202e-05,
 '1001': 5.9189530751007324e-05,
 '0011': 0.9238006439109681,
 '0111': 0.00902511296215247,
 '1011': 0.010357472582643119}

In [126]:
norm_filter_dist = norm_dict(filter_likely_recombined)

In [127]:
norm_filter_dist

{'0001': 0.022566469612573425,
 '0101': 2.0286955367292383e-05,
 '1001': 6.133378871014365e-05,
 '0011': 0.9572671515556102,
 '0111': 0.009352065550822394,
 '1011': 0.01073269253691672}

In [128]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(norm_filtered_lists[i], shots = total_counts(norm_filtered_lists[i]), Pauli = Pauli_tuple[0])  

In [129]:
final_expect_val

(-1.0548950417139107+0j)

In [130]:
import json
with open('LiH_auckland_check0.json', 'w') as f:
    json.dump(norm_filtered_lists, f)

In [ ]:
cals_from_file(self, cals_file)

In [131]:
mit = mthree.M3Mitigation(device)

In [132]:
mit.cals_from_file(cals_file = 'mit_data')

In [134]:
def update_dict(quasis, total_shots = None):
    for key in quasis.keys():
        quasis[key] = quasis[key]*total_shots

In [136]:
expect_val = 0
miti_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        total_shots = total_counts(norm_filtered_lists[i])
        quasis = mit.apply_correction(norm_filtered_lists[i], [16,14,11,8])
        update_dict(quasis, total_shots = total_shots)
        miti_val += coeff * evaluation(quasis, shots = total_shots, Pauli = Pauli_tuple[0]) 
        expect_val += coeff * evaluation(norm_filtered_lists[i], shots = total_shots, Pauli = Pauli_tuple[0]) 

In [137]:
expect_val

(-1.0548950417139107+0j)

In [138]:
miti_val

(-1.0781097866166391+0j)

In [140]:
mit_lists = []
for i in range(0, len(pauli_commute)):
    total_shots = total_counts(norm_filtered_lists[i])
    quasis = mit.apply_correction(norm_filtered_lists[i], [16,14,11,8])
    update_dict(quasis, total_shots = total_shots)
    mit_lists.append(quasis)

In [141]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(mit_lists[i], shots = total_counts(mit_lists[i]), Pauli = Pauli_tuple[0])  

In [142]:
final_expect_val

(-1.0781097866166391+0j)

In [143]:
with open('LiH_auckland_check0_mit.json', 'w') as f:
    json.dump(mit_lists, f)